**Instituto Tecnológico de Costa Rica - TEC**

***Inteligencia Artificial***

*Docente: Kenneth Obando Rodríguez*

---
# Trabajo Corto 4: Árboles de Decisión
---
Estudiantes:
- Gerald Núñez Chavarría - 2021023226
- Sebastián Arroniz Rojas - 2021108521
- Sebastián Bermúdez Acuña - 2021110666

Link del Cuaderno (recuerde configurar el acceso a público):

    
- [Repositorio de GitHub](https://github.com/GeraldNCH/decision_tree_implementation)

    **Nota:** Este trabajo tiene como objetivo promover la comprensión de la materia y su importancia en la elección de algoritmos. Los alumnos deben evitar copiar y pegar directamente información de fuentes externas, y en su lugar, demostrar su propio análisis y comprensión.

### Entrega
Debe entregar un archivo comprimido por el TecDigital, incluyendo un documento pdf con los resultados de los experimentos y pruebas. La fecha de entrega es el jueves 2 de mayo, antes de las 10:00pm.   

Instrucciones:

Las alternativas se rifarán en clase utilizando números aleatorios. Deberá realizar la asignación propuesta. Si realiza ambos ejercicios, recibirá 20 puntos en **la nota porcentual de la actividad**, para aplicar a la totalidad de los puntos extra es necesario que ambas actividades se completen al 100%


## Actividad - Taller

1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha

2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hyperparámetros:
   - max_depth: Cantidad máxima de variables que se pueden explorar
   - min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
   - criterio: función que se utilizará para calcular la impuridad.

3. Divida los datos en los conjuntos tradicionales de entrenamiento y prueba, de forma manual, sin utilizar las utilidades de sklearn (puede utilizar índices de Numpy o Pandas)

4. Implemente una función que se llame `validacion_cruzada` que entrene $k$ modelos y reporte las métricas obtenidasd:
  a. Divida el conjunto de entrenamiento en $k$ subconjuntos excluyentes
  b. Para cada uno de los $k$ modelos, utilice un subconjunto como validación
  c. Reporte la media y la desviación estándar para cada una de las métricas, todo debe realizarse solo usando Numpy:
    - Accuracy
    - Precision
    - Recall
    - F1
  
5. Entrene 10 combinaciones distintas de parámetros para su implementación de Arbol de Decisión y utilizando su implementación de `validacion_cruzada`.

6. Utilizando los resultados obtenidos analice cuál y porqué es el mejor modelo para ser usado en producción.

7. Compruebe las métricas usando el conjunto de prueba y analice el resultado
   


# Implementación
---


In [89]:
import numpy as np
import pandas as pd

## Obtener la data

In [90]:
data = pd.read_csv("../docs/data/Breast_cancer_data.csv")
data.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


### 1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha

In [91]:
class DTNode:
    def __init__(self, feature=None, threshold=None, gini=None, sample_count=None, left=None, right=None, value=None):
        self.feature = feature  # Used for splitting
        self.threshold = threshold  # Threshold for division
        self.gini = gini
        self.sample_count = sample_count  # Number of samples in this node
        self.left = left
        self.right = right

        # for leaf
        self.value = value



### 2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hiperparámetros:
   - max_depth: Cantidad máxima de variables que se pueden explorar
   - min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
   - criterio: función que se utilizará para calcular la impuridad.

In [148]:
class DecisionTree:
    def __init__(self, max_depth, min_split_samples, criterion='gini'):
        self.max_depth = max_depth
        self.min_split_samples = min_split_samples 
        self.criterion = criterion # gini or entropy

        self.root = None  # Root of tree

    """Recursive function to build the tree"""
    def buildTree(self, dataset, currDepth=0):
        X, Y = dataset[:, :-1], dataset[:, -1] # Splitting the features and target variable
        numSamples, numFeatures = np.shape(X)

        # Split until stopping conditions are met
        if numSamples >= self.min_split_samples and currDepth <= self.max_depth:
            bestSplit = self.getBestSplit(dataset, numFeatures)

            if bestSplit[self.criterion] > 0: # To make sure to not split a pure node

                # Left subtree
                leftSubtree = self.buildTree(bestSplit['dataset_left'], currDepth + 1)

                # Right subtree
                rightSubtree = self.buildTree(bestSplit['dataset_right'], currDepth + 1)

                # Return node
                return DTNode(bestSplit['feature'],
                              bestSplit['threshold'],
                              bestSplit[self.criterion],
                              numSamples,
                              leftSubtree,
                              rightSubtree)
            
        leafValue = self.calculateLeafValue(Y) # If stopping conditions are met, return a leaf node
        return DTNode(value=leafValue)
    
    """Find the best split from all the features"""
    def getBestSplit(self, dataset, numFeatures):
        bestSplit = {} # To store the best split info
        maxInfoGain = -float('inf')

        # Loop through all features
        for featureIdx in range(numFeatures):
            featureValues = dataset[:, featureIdx]
            possibleThresholds = np.unique(featureValues)

            # Loop through all unique values of the feature
            for threshold in possibleThresholds:
                datasetLeft, datasetRight = self.split(dataset, featureIdx, threshold) # Split the dataset

                if len(datasetLeft) > 0 and len(datasetRight) > 0: # Make sure it is not empty
                    y, leftY, rightY = dataset[:, -1], datasetLeft[:, -1], datasetRight[:, -1] # Extracting targets
                    infoGain = self.calculateInfoGain(y, leftY, rightY)

                    # Update the best split
                    if infoGain > maxInfoGain:
                        bestSplit['feature'] = featureIdx
                        bestSplit['threshold'] = threshold
                        bestSplit[self.criterion] = infoGain
                        bestSplit['dataset_left'] = datasetLeft
                        bestSplit['dataset_right'] = datasetRight

                        maxInfoGain = infoGain

        return bestSplit
    
    """Split the dataset into left and right subtrees"""
    def split(self, dataset, featureIdx, threshold):
        datasetLeft = np.array([row for row in dataset if row[featureIdx] <= threshold]) # Left subtree, less than or equal to threshold
        datasetRight = np.array([row for row in dataset if row[featureIdx] > threshold]) # Right subtree, greater than threshold
        return datasetLeft, datasetRight
    
    """Calculate the info_gain value, corresponding to the worth of a trio of parent and its possible childs"""
    def calculateInfoGain(self, parent, lChild, rChild):
        # Importance (weight) of each child in respect to their parent
        weight_l = len(lChild) / len(parent) 
        weight_r = len(rChild) / len(parent)

        if self.criterion == 'gini':
            return self.calculateGini(parent) - (weight_l * self.calculateGini(lChild) + weight_r * self.calculateGini(rChild))
        else:
            return self.calculateCriterion(parent) - (weight_l * self.calculateCriterion(lChild) + weight_r * self.calculateCriterion(rChild))
    
    """Calculate the gini impurity"""
    def calculateGini(self, y):
        classes = np.unique(y)
        gini = 0
        for cls in classes:
            p = len(y[y == cls]) / len(y)
            gini += p * (1 - p)
        return 1 - gini
    
    def calculateCriterion(self, y):
        # Not Implemented
        pass
    
    """Calculate the most common value in the leaf"""
    def calculateLeafValue(self, y):
        Y = list(y)
        return max(Y, key=Y.count) # Return the most common value in the leaf
    
    """Train the tree"""
    def fit(self, X, Y):
        Y = np.expand_dims(Y, axis=1)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.buildTree(dataset)
    
    """Predict a new data set"""
    def predict(self, X):
        preditions = [self.makePrediction(x, self.root) for x in X]
        return preditions
    def makePrediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.makePrediction(x, tree.left)
        else:
            return self.makePrediction(x, tree.right)

### 3. Divida los datos en los conjuntos tradicionales de entrenamiento y prueba, de forma manual, sin utilizar las utilidades de sklearn (puede utilizar índices de Numpy o Pandas)

In [149]:
trainRatio = 0.6  # Ratio to divide data.
trainSize = int(trainRatio * len(data))

trainData = data.iloc[:trainSize]  # Set for training
testData = data.iloc[trainSize:]  # Set for testing

# Separate features (X) and labels (Y) in each set
X_train = trainData.drop(columns=['diagnosis'])
Y_train = trainData['diagnosis']

X_test = testData.drop(columns=['diagnosis'])
Y_test = testData['diagnosis']

trainData.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


### 4. Implemente una función que se llame `validacion_cruzada` que entrene $k$ modelos y reporte las métricas obtenidasd:
  a. Divida el conjunto de entrenamiento en $k$ subconjuntos excluyentes
  b. Para cada uno de los $k$ modelos, utilice un subconjunto como validación
  c. Reporte la media y la desviación estándar para cada una de las métricas, todo debe realizarse solo usando Numpy:
    - Accuracy
    - Precision
    - Recall
    - F1

In [152]:
def crossValidation(model, X_train, Y_train, k=5):
    foldSize = len(X_train) // k  # Size of each fold
    metrics = {'accuracy': [], 'precision': [], 'recall': []}  # To store the metrics for each fold

    for i in range(0, len(X_train), foldSize):
        # Test and Train set for this fold
        X_valuesFold = X_train[i:i + foldSize]
        Y_valuesFold = Y_train[i:i + foldSize]
        X_trainFold = np.concatenate((X_train[:i], X_train[i + foldSize:])) 
        Y_trainFold = np.concatenate((Y_train[:i], Y_train[i + foldSize:]))

        model.fit(X_trainFold, Y_trainFold)  # Train the model
        predictions = model.predict(X_valuesFold)  # Predict the test set

        # Calculate the metrics
        accuracy = np.mean(predictions == Y_valuesFold)
        precision = np.sum((predictions == 1) & (Y_valuesFold == 1)) / np.sum(predictions == 1)
        recall = np.sum((predictions == 1) & (Y_valuesFold == 1)) / np.sum(Y_valuesFold == 1)

        # Store metrics
        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)

    meanMetrics = {metric: np.mean(values) for metric, values in metrics.items()}
    stdMetrics = {metric: np.std(values) for metric, values in metrics.items()}

    return meanMetrics, stdMetrics

### 5. Entrene 10 combinaciones distintas de parámetros para su implementación de Arbol de Decisión y utilizando su implementación de `validacion_cruzada`.

In [153]:
def trainModel(X_train, Y_train):
    params = [
        {'max_depth': 3, 'min_split_samples': 2},
        {'max_depth': 5, 'min_split_samples': 2},
        {'max_depth': 2, 'min_split_samples': 5},
        {'max_depth': 3, 'min_split_samples': 3},
        {'max_depth': 2, 'min_split_samples': 2},
        {'max_depth': 5, 'min_split_samples': 3},
        {'max_depth': 3, 'min_split_samples': 5},
        {'max_depth': 5, 'min_split_samples': 5},
    ]

    for i, params in enumerate(params):
        print(f"Training model {i + 1} with params: {params}")
        
        tree = DecisionTree(params['max_depth'], params['min_split_samples'])
        meanMetrics, stdMetrics = crossValidation(tree, X_train, Y_train, k=5) # Get cross validation metrics
        
        # Imprimir las métricas obtenidas
        print(f"Metrics for model {i + 1}:")
        for metric, value in meanMetrics.items():
            print(f"{metric}: {value:.4f} (±{stdMetrics[metric]:.4f})")
        print("-"*20)

trainModel(X_train.to_numpy(), Y_train.to_numpy())

Training model 1 with params: {'max_depth': 3, 'min_split_samples': 2}


/var/folders/tz/06nn_cjs5nxc2rkr6mvzdxx00000gn/T/ipykernel_30188/3008803668.py:17: RuntimeWarning: invalid value encountered in scalar divide
  precision = np.sum((predictions == 1) & (Y_valuesFold == 1)) / np.sum(predictions == 1)


Metrics for model 1:
accuracy: 0.5368 (±0.2475)
precision: nan (±nan)
recall: 0.0000 (±0.0000)
--------------------
Training model 2 with params: {'max_depth': 5, 'min_split_samples': 2}
Metrics for model 2:
accuracy: 0.5368 (±0.2475)
precision: nan (±nan)
recall: 0.0000 (±0.0000)
--------------------
Training model 3 with params: {'max_depth': 2, 'min_split_samples': 5}
Metrics for model 3:
accuracy: 0.5368 (±0.2475)
precision: nan (±nan)
recall: 0.0000 (±0.0000)
--------------------
Training model 4 with params: {'max_depth': 3, 'min_split_samples': 3}
Metrics for model 4:
accuracy: 0.5368 (±0.2475)
precision: nan (±nan)
recall: 0.0000 (±0.0000)
--------------------
Training model 5 with params: {'max_depth': 2, 'min_split_samples': 2}
Metrics for model 5:
accuracy: 0.5368 (±0.2475)
precision: nan (±nan)
recall: 0.0000 (±0.0000)
--------------------
Training model 6 with params: {'max_depth': 5, 'min_split_samples': 3}
Metrics for model 6:
accuracy: 0.5368 (±0.2475)
precision: nan (±

### 6. Utilizando los resultados obtenidos analice cuál y porqué es el mejor modelo para ser usado en producción.

### 7. Compruebe las métricas usando el conjunto de prueba y analice el resultado

## Rúbrica para la Implementación de un Árbol de Decisión

**Nota: Esta rúbrica se basa en la calidad de la implementación y los resultados obtenidos, no en la cantidad de código.**

**1. Creación de la Clase Nodo (10 puntos)**

- [ ] Se crea una clase `Nodo` con los atributos mencionados en las especificaciones (feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha).
- [ ] Los atributos se definen correctamente y se asignan de manera apropiada.

**2. Creación de la Clase Árbol de Decisión (20 puntos)**

- [ ] Se crea una clase que implementa un árbol de decisión.
- [ ] La clase utiliza las funciones presentadas en el cuaderno.
- [ ] Se implementan los hyperparámetros solicitados (max_depth, min_split_samples, criterio).
- [ ] La clase es capaz de entrenar un árbol de decisión con los hyperparámetros especificados.

**3. División de Datos (10 puntos)**

- [ ] Los datos se dividen en conjuntos de entrenamiento y prueba de forma manual.
- [ ] Se utiliza Numpy o Pandas para realizar esta división.
- [ ] Se garantiza que los conjuntos sean excluyentes.

**4. Implementación de Validación Cruzada (20 puntos)**

- [ ] Se implementa la función `validacion_cruzada` correctamente.
- [ ] Los datos de entrenamiento se dividen en k subconjuntos excluyentes.
- [ ] Se entrena y evalúa un modelo para cada subconjunto de validación.
- [ ] Se calculan y reportan las métricas de accuracy, precision, recall y F1.
- [ ] Se calcula la media y la desviación estándar de estas métricas.

**5. Entrenamiento de Modelos (20 puntos)**

- [ ] Se entrenan 10 combinaciones distintas de parámetros para el árbol de decisión.
- [ ] Cada combinación se entrena utilizando la función `validacion_cruzada`.
- [ ] Los resultados de las métricas se registran adecuadamente.

**6. Análisis de Modelos (10 puntos)**

- [ ] Se analizan los resultados obtenidos y se selecciona el mejor modelo para ser utilizado en producción.
- [ ] Se proporciona una justificación clara y fundamentada sobre por qué se eligió ese modelo.

**7. Prueba en el Conjunto de Prueba (10 puntos)**

- [ ] Se comprueban las métricas del modelo seleccionado utilizando el conjunto de prueba.
- [ ] Se analizan los resultados y se comentan las conclusiones.

**General (10 puntos)**

- [ ] El código se documenta de manera adecuada, incluyendo comentarios que expliquen las secciones clave.
- [ ] El código se ejecuta sin errores y sigue buenas prácticas de programación.
- [ ] La presentación de los resultados es clara y fácil de entender.
- [ ] Se cumple con todos los requisitos y las especificaciones proporcionadas.

**Puntuación Total: 100 puntos**

